In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import models
from tqdm import tqdm

In [7]:
# Трансформация: преобразуем изображения в тензоры и нормализуем их
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Нормализация изображений
])

In [8]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
# Модель ResNet-18
model = models.resnet18(pretrained=True)

# Изменим последний слой, чтобы соответствовать 10 классам Fashion MNIST
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 10)

# Перемещаем модель на доступное устройство (GPU, если оно есть)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Определим функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()  # Для многоклассовой классификации
optimizer = optim.Adam(model.parameters(), lr=0.001)

c:\Users\Zalman\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Zalman\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
print(device)

cuda


In [12]:
import sys

print(torch.__version__)
print(sys.version)

2.4.1+cu124
3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]


In [13]:
print(torch.cuda.is_available())

True


In [18]:
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()  # Устанавливаем модель в режим обучения
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        # Используем tqdm для отслеживания прогресса
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}')
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()  # Обнуляем градиенты
            outputs = model(images)  # Прогоняем данные через модель
            loss = criterion(outputs, labels)  # Вычисляем ошибку
            loss.backward()  # Обратное распространение ошибки
            optimizer.step()  # Обновляем веса
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Обновляем описание прогресс-бара
            progress_bar.set_postfix({'Loss': f'{loss.item():.4f}', 'Accuracy': f'{100 * correct / total:.2f}%'})

        print(f"Epoch {epoch + 1}/{epochs} completed. Loss: {running_loss / len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

In [15]:
# Функция для оценки модели на тестовом наборе
def evaluate_model(model, test_loader):
    model.eval()  # Переводим модель в режим оценки
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [19]:
# Обучаем модель
train_model(model, train_loader, criterion, optimizer, epochs=5)

Epoch 1/5: 100%|██████████| 938/938 [00:33<00:00, 28.02it/s, Loss=0.2546, Accuracy=82.70%]


Epoch 1/5 completed. Loss: 0.4870, Accuracy: 82.70%


Epoch 2/5: 100%|██████████| 938/938 [00:32<00:00, 28.55it/s, Loss=0.1322, Accuracy=87.95%]


Epoch 2/5 completed. Loss: 0.3359, Accuracy: 87.95%


Epoch 3/5: 100%|██████████| 938/938 [00:32<00:00, 28.58it/s, Loss=0.2615, Accuracy=89.55%]


Epoch 3/5 completed. Loss: 0.2945, Accuracy: 89.55%


Epoch 4/5: 100%|██████████| 938/938 [00:32<00:00, 28.60it/s, Loss=0.3661, Accuracy=90.50%]


Epoch 4/5 completed. Loss: 0.2615, Accuracy: 90.50%


Epoch 5/5: 100%|██████████| 938/938 [00:32<00:00, 28.64it/s, Loss=0.3140, Accuracy=91.40%]

Epoch 5/5 completed. Loss: 0.2357, Accuracy: 91.40%


In [20]:

# Оценим модель на тестовых данных
evaluate_model(model, test_loader)

Test Accuracy: 90.39%
